In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
df = pd.read_csv("./../churn_data.csv")

df = pd.get_dummies(df, columns=['internationalplan', 'voicemailplan'], prefix = ['internationalplan', 'voicemailplan'])
df = df.dropna()

X = df.drop('churn', axis=1).to_numpy()
y = df.loc[:, 'churn'].to_numpy()

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

df.describe(include="all")

,churn,accountlength,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes
count,4296,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000,4296.000000
unique,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,3690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,100.242480,7.702747,180.212987,100.165736,30.629260,200.689991,100.203445,17.058841,200.298440,99.825885,9.013566,10.257076,4.438315,2.771096,1.570764,0.895717,0.094274,0.726490,0.262803
std,NaN,39.959503,13.444106,53.966046,19.859999,9.167727,50.860432,19.884827,4.323141,50.286335,19.943283,2.262881,2.757205,2.453462,0.743208,1.308207,0.305663,0.292243,0.445812,0.440207
min,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,73.000000,0.000000,143.575000,87.000000,24.410000,165.900000,87.000000,14.100000,167.075000,86.750000,7.520000,8.500000,3.000000,2.300000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,NaN,100.000000,0.000000,180.050000,100.000000,30.600000,200.800000,100.000000,17.070000,200.250000,100.000000,9.010000,10.300000,4.000000,2.783676,1.000000,1.000000,0.000000,1.000000,0.000000
75%,NaN,127.000000,17.000000,216.000000,113.000000,36.720000,234.225000,114.000000,19.912500,234.200000,113.000000,10.540000,12.000000,6.000000,3.240000,2.000000,1.000000,0.000000,1.000000,1.000000


In [ ]:
svm = SVC(kernel='linear', C=100)
svm.fit(Xtrain, ytrain)